In [1]:
import boto3
import json
session = boto3.Session(profile_name='css-lab1')
br_client = session.client("bedrock", region_name="us-east-1")
br_agent_client = session.client("bedrock-agent", region_name="us-east-1")

In [10]:
list(filter(lambda e: 
            # "embed" in e["modelId"].lower() and 
            # "titan" in e["modelId"].lower() and 
            # "v2" in e["modelId"].lower()
            "claude-3-sonnet" in e["modelId"].lower() and 
            "ON_DEMAND" in e["inferenceTypesSupported"]
            , br_client.list_foundation_models()["modelSummaries"]
            ))

[{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0',
  'modelId': 'anthropic.claude-3-sonnet-20240229-v1:0',
  'modelName': 'Claude 3 Sonnet',
  'providerName': 'Anthropic',
  'inputModalities': ['TEXT', 'IMAGE'],
  'outputModalities': ['TEXT'],
  'responseStreamingSupported': True,
  'customizationsSupported': [],
  'inferenceTypesSupported': ['ON_DEMAND'],
  'modelLifecycle': {'status': 'ACTIVE'}}]

In [68]:
# use bedrock client to list storage
result = br_agent_client.list_knowledge_bases()["knowledgeBaseSummaries"]
kb_id = result[0]["knowledgeBaseId"]
kb_id, result

('WQOP6BE3H0',
 [{'description': 'Pinecone index for recipes',
   'knowledgeBaseId': 'WQOP6BE3H0',
   'name': 'TestKnowledgeBase',
   'status': 'ACTIVE',
   'updatedAt': datetime.datetime(2024, 7, 11, 0, 29, 25, 258237, tzinfo=tzutc())}])

In [69]:
result = br_agent_client.list_data_sources(knowledgeBaseId=kb_id)["dataSourceSummaries"]
ds_id = result[0]["dataSourceId"]
ds_id, result

('WBFWRLQGT8',
 [{'dataSourceId': 'WBFWRLQGT8',
   'description': 'Data source for recipes',
   'knowledgeBaseId': 'WQOP6BE3H0',
   'name': 'TestDataSource',
   'status': 'AVAILABLE',
   'updatedAt': datetime.datetime(2024, 7, 11, 0, 29, 38, 900310, tzinfo=tzutc())}])

In [80]:
job = br_agent_client.start_ingestion_job(dataSourceId=ds_id, knowledgeBaseId=kb_id)
job["ingestionJob"]

{'dataSourceId': 'WBFWRLQGT8',
 'ingestionJobId': 'HLPSDVO8RT',
 'knowledgeBaseId': 'WQOP6BE3H0',
 'startedAt': datetime.datetime(2024, 7, 11, 0, 52, 19, 635442, tzinfo=tzutc()),
 'statistics': {'numberOfDocumentsDeleted': 0,
  'numberOfDocumentsFailed': 0,
  'numberOfDocumentsScanned': 0,
  'numberOfMetadataDocumentsModified': 0,
  'numberOfMetadataDocumentsScanned': 0,
  'numberOfModifiedDocumentsIndexed': 0,
  'numberOfNewDocumentsIndexed': 0},
 'status': 'STARTING',
 'updatedAt': datetime.datetime(2024, 7, 11, 0, 52, 19, 635442, tzinfo=tzutc())}

In [81]:
import time


job_id = job["ingestionJob"]["ingestionJobId"]
while True:
    job_status = br_agent_client.get_ingestion_job(ingestionJobId=job_id, knowledgeBaseId=kb_id, dataSourceId=ds_id)
    print(job_status["ingestionJob"]["status"])
    time.sleep(1)

IN_PROGRESS
IN_PROGRESS
IN_PROGRESS
IN_PROGRESS
IN_PROGRESS
IN_PROGRESS
IN_PROGRESS
IN_PROGRESS
IN_PROGRESS
IN_PROGRESS
IN_PROGRESS
IN_PROGRESS
IN_PROGRESS
IN_PROGRESS
IN_PROGRESS
IN_PROGRESS
IN_PROGRESS
IN_PROGRESS
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE


KeyboardInterrupt: 

In [38]:

json.loads(job_status["ingestionJob"]["failureReasons"][0])

['Encountered error: Ignored 1 files as their file format was not supported. [Files: s3://bedrockagentsstack-testing-recipebucket7d535f14-1zxwom0f1fj0/.DS_Store]. Call to Amazon S3 Source did not succeed.',
 'The server encountered an internal error while processing the request.',
 'Encountered error: The vector database encountered an error while processing the request.. Issue occurred while processing file: s3://bedrockagentsstack-testing-recipebucket7d535f14-1zxwom0f1fj0/creamy garden herb chicken.pdf. Call to Pinecone Vector Database did not succeed.',
 'Encountered error: Ignored 1 files as their file format was not supported. [Files: s3://bedrockagentsstack-testing-recipebucket7d535f14-1zxwom0f1fj0/.DS_Store]. Call to Amazon S3 Source did not succeed.',
 'The server encountered an internal error while processing the request.',
 'Encountered error: The vector database encountered an error while processing the request.. Issue occurred while processing file: s3://bedrockagentsstack-

In [7]:
agent_id = br_agent_client.list_agents()["agentSummaries"][0]["agentId"]
agent_version = br_agent_client.list_agent_versions(agentId=agent_id)["agentVersionSummaries"][0]["agentVersion"]

In [18]:
resp = br_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion = agent_version,
    actionGroupName="test-action-group",
    parentActionGroupSignature="AMAZON.CodeInterpreter",
    actionGroupState="ENABLED"
)
action_group_id = resp["agentActionGroup"]["actionGroupId"]
action_group_id


'ORWZTGALHQ'

In [19]:
br_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion = agent_version,
    actionGroupId=action_group_id,
    actionGroupName="test-action-group",
    parentActionGroupSignature="AMAZON.CodeInterpreter",
    actionGroupState="DISABLED"
)
resp = br_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion = agent_version,
    actionGroupId=action_group_id,
    # actionGroupName="test-action-group",
    # parentActionGroupSignature="AMAZON.CodeInterpreter",
    # actionGroupState="DISABLED"
)
resp

{'ResponseMetadata': {'RequestId': 'e62a4b14-0dd2-4e97-81a6-1058b622bff5',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'date': 'Sun, 14 Jul 2024 14:35:43 GMT',
   'content-type': 'application/json',
   'content-length': '0',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e62a4b14-0dd2-4e97-81a6-1058b622bff5',
   'x-amz-apigw-id': 'a6BJ6FZZoAMEK7g=',
   'x-amzn-trace-id': 'Root=1-6693e23f-0e80f4dd493407a537d56b09'},
  'RetryAttempts': 0}}

In [12]:
resp = br_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion = agent_version,
    actionGroupId=action_group_id,
    actionGroupName="test-action-group",
    parentActionGroupSignature="AMAZON.CodeInterpreter",
    actionGroupState="ENABLED"
)
resp

{'ResponseMetadata': {'RequestId': 'b0c5cc0b-5b29-4874-a237-a2b61b606cb6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 14 Jul 2024 14:33:03 GMT',
   'content-type': 'application/json',
   'content-length': '356',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b0c5cc0b-5b29-4874-a237-a2b61b606cb6',
   'x-amz-apigw-id': 'a6Aw6HnxoAMEUdw=',
   'x-amzn-trace-id': 'Root=1-6693e19f-3602dadf37a1820f377e3053'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupId': 'KG6QNK8PUJ',
  'actionGroupName': 'test-action-group',
  'actionGroupState': 'ENABLED',
  'agentId': 'ZTDMPOKWSS',
  'agentVersion': 'DRAFT',
  'clientToken': 'a8c0301d-a4d2-4d0b-aff7-933117b88d55',
  'createdAt': datetime.datetime(2024, 7, 14, 14, 30, 26, 578767, tzinfo=tzutc()),
  'parentActionSignature': 'AMAZON.CodeInterpreter',
  'updatedAt': datetime.datetime(2024, 7, 14, 14, 33, 3, 335036, tzinfo=tzutc())}}